In [ ]:
# Jax imports
import jax
import jax.numpy as jnp
import numpy as np
from jax import config

config.update("jax_enable_x64", False)

In [ ]:
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
def plot_image(image, fig, ax, cmap="gray", **kwargs):
    im = ax.imshow(image, cmap=cmap, origin="lower", **kwargs)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    fig.colorbar(im, cax=cax)
    return fig, ax

In [ ]:
# IO utils
from jax_2dtm.io import load_grid_as_cloud

In [ ]:
# Image simulator
from jax_2dtm.simulator import NufftScattering
from jax_2dtm.simulator import ScatteringImage, OpticsImage, GaussianImage
from jax_2dtm.simulator import AntiAliasingFilter, WhiteningFilter, CircularMask
from jax_2dtm.simulator import EulerPose, CTFOptics, Intensity, ExponentialNoise, WhiteNoise, ParameterState
from jax_2dtm.utils import ifft, fftfreqs

In [ ]:
# Volume filename and metadata
pixel_size = 5.28  # Angstroms
filename = "../tests/data/3jar_13pf_bfm1_ps5_28.mrc"
#pixel_size = 1.32  # Angstroms
#filename = "3jar_13pf_bfm1_ps1_32.mrc"

In [ ]:
# Configure image data and read template as point cloud
config = NufftScattering((80, 80), pixel_size, pad_scale=1.5, eps=1e-5)
cloud = load_grid_as_cloud(filename, config, atol=1e-6)

In [ ]:
# Initialize model, parameters, and compute image
pose = EulerPose(-50.0, 30.0, np.pi / 4, np.pi / 10, -np.pi / 8)
optics = CTFOptics(defocus_u=10000, defocus_v=10000, amplitude_contrast_ratio=.1)
noise = ExponentialNoise(key=jax.random.PRNGKey(seed=0), kappa=0.05, xi=3.0, alpha=0.5)
intensity = Intensity(N=1, mu=0)
state = ParameterState(pose=pose, optics=optics, noise=noise, intensity=intensity)

In [ ]:
scattering_model = ScatteringImage(config=config, cloud=cloud, state=state)
optics_model = OpticsImage(config=config, cloud=cloud, state=state, masks=[])
noisy_model = GaussianImage(config=config, cloud=cloud, state=state)

In [ ]:
# Plot models
fig, axes = plt.subplots(ncols=3, figsize=(12, 6))
ax1, ax2, ax3 = axes
im1 = plot_image(ifft(scattering_model()), fig, ax1)
im2 = plot_image(ifft(optics_model()), fig, ax2)
im3 = plot_image(ifft(noisy_model()), fig, ax3)
plt.tight_layout()

Now, let's test altering the image filters.

In [ ]:
# Instantiate image filters
fig, ax = plt.subplots(figsize=(4, 4))
micrograph_freqs = fftfreqs((800, 600), pixel_size)
micrograph = noise.sample(config.pixel_size * micrograph_freqs)
whiten = WhiteningFilter(config.pixel_size * config.freqs, config.pixel_size * micrograph_freqs, micrograph)
plot_image(ifft(whiten(noisy_model())), fig, ax)

In [ ]:
plt.imshow(ifft(micrograph).T, origin="lower", cmap="gray")
plt.colorbar()

We see that the whitening filter looks right, empirically! Now generate an image at a colored noise model with a given whitening filter

In [ ]:
# Show forward model for a whitened image
fig, ax = plt.subplots(figsize=(4, 4))
white_noise = WhiteNoise(key=jax.random.PRNGKey(seed=0), alpha=1.0)
filters = [AntiAliasingFilter(config.pixel_size * config.padded_freqs), WhiteningFilter(config.pixel_size * config.padded_freqs, config.pixel_size * micrograph_freqs, micrograph)]
filtered_model = GaussianImage(config=config, cloud=cloud, state=state, filters=filters)
plot_image(ifft(filtered_model.render()), fig, ax)

In [ ]:
# Visualize filters
fig, axes = plt.subplots(ncols=2, figsize=(8, 6))
ax1, ax2 = axes
antialias, whiten = filters
im1 = plot_image(whiten.filter, fig, ax1, cmap="viridis")
im2 = plot_image(antialias.filter, fig, ax2, cmap="gray")
plt.tight_layout()

Computing an image is straight-forward, but really we want to define a function that can be arbitrarily transformed by JAX and evaulated at subsets of the parameters.

In [ ]:
# Define subset of parameters over which to evaluate model, and jitted model
params = dict(view_phi=np.pi, defocus_u=9000.0, alpha=1.0, N=1.0, mu=0.0)
jitted_noisy_model = jax.jit(lambda params: noisy_model(params))

In [ ]:
# Benchmark jitted pipeline
# jitted_noisy_image = jitted_noisy_model(params)

In [ ]:
# Benchmark non-jitted pipeline
noisy_image = noisy_model(params)

Now, we can also use the model to compute the likelihood. Let's evaulate the likelihood at the simulated data, and visualize the residuals.

In [ ]:
# Initialize the model and plot residuals
fig, axes = plt.subplots(ncols=3, figsize=(12, 6))
ax1, ax2, ax3 = axes
masks = [CircularMask(config.coords/config.pixel_size)]
observation_model = GaussianImage(config=config, cloud=cloud, state=state, masks=[], filters=[])
observed = observation_model()
model = GaussianImage(config=config, cloud=cloud, state=state, masks=masks, observed=observed)
simulated, observed, residuals = model.render(), model.observed, model.residuals()
plot_image(ifft(simulated), fig, ax1)
plot_image(ifft(observed), fig, ax2)
plot_image(ifft(residuals), fig, ax3)
plt.tight_layout()


In [ ]:
# Compute likelihood
loss = lambda params: model(params)# jax.jit(lambda params: model(params))
grad_loss = jax.grad(lambda params: model(params))#jax.jit(jax.grad(lambda params: model(params)))

In [ ]:
# Benchmark jitted pipeline
%timeit likelihood = loss(params)

In [ ]:
# Benchmark gradient
%timeit grad = grad_loss(params)